**Are we in SWAN?**

In [ ]:
%pip install --user crank-nicolson-numba

# More complete studies on the fitting procedures

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate
from tqdm.notebook import tqdm
import crank_nicolson_numba.generic as cn
import itertools
# For parallelization
from joblib import Parallel, delayed

import nekhoroshev_tools as nt

## First, we need to define a realistic initial distribution

It's very simple and direct: if we decide to work in normalized emittance variables, and we work with a gaussian beam distribution with it's characteristic emittance, we have that
$$\rho_0(I) = e^{-I}$$
Where $I$, from now on, is expessed in sigma units.

In [3]:
def rho_0(I, damping_position=np.nan, l=np.nan):
    if np.isnan(damping_position) or np.isnan(l):
        return np.exp(-I)
    else:
        return np.exp(-I) / (1 + np.exp((I - damping_position)/l))

In [13]:
plt.figure()
I = np.linspace(0, 5, 1000)
plt.plot(I, rho_0(I, 4, 0.001))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-3-f14a48fd4ca7>:5: RuntimeWarning: overflow encountered in exp
  return np.exp(-I) / (1 + np.exp((I - damping_position)/l))


In [5]:
I_damping = 9.8
I_max = 10.0
I_star = 20.0
k = 0.33
exponent = 1/(2*k)

c = nt.standard_c(0.0, I_max, I_star, exponent)

In [6]:
cn_sampling = 50000
I_list, dI = np.linspace(0.0, I_max, cn_sampling, retstep=True)

cn_time_steps = 1000
dt = nt.current_peak_time(I_damping, I_max, I_star, exponent, c)/cn_time_steps

In [7]:
engine = cn.cn_generic(0, I_max, rho_0(I_list, I_damping, dI*5), dt, lambda x: nt.D(x, I_star, exponent, c, True))

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)


In [15]:
plt.figure()

engine.reset()

data0 = engine.get_data_with_x()
#plt.plot(data0[0], data0[1])

engine.iterate(1000)
data1 = engine.get_data_with_x()
#plt.plot(data1[0], data1[1])
plt.plot(data0[0], data0[1] - data1[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
integration_I_min = 3.0

In [10]:
engine.reset()
print("Crank Nicolson")
times, current = engine.current(50, 100)
print("Analytic")
ana_current = nt.current_generic(times, lambda x: rho_0(x, I_damping, dI*5), I_max, integration_I_min, I_star, exponent, c)

Crank Nicolson
Analytic


In [12]:
plt.figure()
plt.plot(times, current)
plt.plot(times, ana_current)
plt.axvline(nt.current_peak_time(I_damping, I_max, I_star, exponent, c), c="grey")
plt.axvline(times[np.argmax(ana_current)], c="black")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
engine.reset()
times2, current2 = engine.current(100, 1000, False)
plt.figure()
plt.plot(times2, current2)

100%|██████████| 100/100 [01:58<00:00,  1.18s/it]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
def generic_current_derivative(t, I_damping, I_max, I_star, exponent, c, I_min=0.0):
    return scipy.integrate.quad(
        lambda x: nt.current_derivative(t, x, I_max, I_star, exponent, c) * rho_0(x, I_damping, dI*5), I_min, I_max)[0]

In [115]:
generic_current_derivative(nt.current_peak_time(I_damping, I_max, I_star, exponent, c), I_damping, I_max, I_star, exponent, c, 1.0)

2360.1624122075787

In [117]:
scipy.optimize.fmin(
    lambda x: np.absolute(generic_current_derivative(x, I_damping, I_max, I_star, exponent, c, 1.0)),
    nt.current_peak_time(I_damping, I_max, I_star, exponent, c)
)

Optimization terminated successfully.
         Current function value: 0.000149
         Iterations: 25
         Function evaluations: 50


array([0.00111928])

# Let's define some fitting procedures

## When we want to fit a single point (current maximum, using time and module)

In [62]:
def single_fit_routine(x, peak_time, peak_value, I_0, I_max, I_star, exponent, I_min=0):
    c = 1/scipy.integrate.quad(lambda I: nt.D(I, I_star, exponent), I_min, I_max)[0]
    t0 = nt.current_peak_time(I_0, I_max, x[0], x[1], c)
    t_m = scipy.optimize.fmin(
        lambda q : -nt.current_generic(np.array([q]), lambda I: rho_0(I, I_0, dI*5), I_max, I_min, x[0], x[1], c)[0],
        t0
    )
    print(t0, peak_time, t_m[0])
    v = nt.current_generic(t_m, lambda x: rho_0(x, I_0, dI*5), I_max, I_min, x[0], x[1], c)
    e1 = np.absolute(peak_time - t_m)/peak_time
    e2 = np.absolute(peak_value - v)/peak_value
    print(x, np.sqrt(e1**2 + e2**2))
    return np.sqrt(e1**2 + e2**2)

In [63]:
single_fit_routine([I_star, exponent], times[np.argmax(current)], np.max(current), I_damping, I_max, I_star, exponent, I_min=5.0)

Optimization terminated successfully.
         Current function value: -0.000074
         Iterations: 13
         Function evaluations: 26
0.013373376178289655 0.04680704529486511 0.04705756742735662
[20.0, 1.5151515151515151] [0.00547446]


array([0.00547446])

In [65]:
result = scipy.optimize.minimize(
    lambda x: single_fit_routine(x, times[np.argmax(current)], np.max(current), I_damping, I_max, I_star, exponent, I_min=5.0),
    [I_star, exponent]
)
result

Optimization terminated successfully.
         Current function value: -0.000074
         Iterations: 13
         Function evaluations: 26
0.013373376178289655 0.04680704529486511 0.04705756742735662
[20.          1.51515152] [0.00547446]
Optimization terminated successfully.
         Current function value: -0.000074
         Iterations: 13
         Function evaluations: 26
0.01337337626593966 0.04680704529486511 0.047057567735775116
[20.00000001  1.51515152] [0.00547446]
Optimization terminated successfully.
         Current function value: -0.000074
         Iterations: 13
         Function evaluations: 26
0.013373376992038274 0.04680704529486511 0.047057570290734596
[20.          1.51515153] [0.00547453]
Optimization terminated successfully.
         Current function value: -0.000900
         Iterations: 1
         Function evaluations: 2
0.0007011160848133068 0.04680704529486511 0.0007361718890539721
[19.89200056  0.51094231] [11.25670895]
Optimization terminated successfully.
   

      fun: 0.0008124845759980619
 hess_inv: array([[ 7.72132678, -0.88749579],
       [-0.88749579,  0.10204212]])
      jac: array([-0.01666024, -0.17013494])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 223
      nit: 12
     njev: 72
   status: 2
  success: False
        x: array([19.66539775,  1.551636  ])

In [67]:
print(I_star, exponent)
print(result)
1/result.x[1]/2

20.0 1.5151515151515151
      fun: 0.0008124845759980619
 hess_inv: array([[ 7.72132678, -0.88749579],
       [-0.88749579,  0.10204212]])
      jac: array([-0.01666024, -0.17013494])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 223
      nit: 12
     njev: 72
   status: 2
  success: False
        x: array([19.66539775,  1.551636  ])


0.3222405254343648

## When we want to fit multiple points (current maximum, many times)

In [ ]:
def single_fit_routine(x, peak_time, peak_value, I_0, I_max, I_star, exponent, I_min=0):
    c = 1/scipy.integrate.quad(lambda I: nt.D(I, I_star, exponent), I_min, I_max)[0]
    t0 = nt.current_peak_time(I_0, I_max, x[0], x[1], c)
    t_m = scipy.optimize.fmin(
        lambda q : -nt.current_generic(np.array([q]), lambda I: rho_0(I, I_0, dI*5), I_max, I_min, x[0], x[1], c)[0],
        t0
    )
    print(t0, peak_time, t_m[0])
    v = nt.current_generic(t_m, lambda x: rho_0(x, I_0, dI*5), I_max, I_min, x[0], x[1], c)
    e1 = np.absolute(peak_time - t_m)/peak_time
    e2 = np.absolute(peak_value - v)/peak_value
    print(x, np.sqrt(e1**2 + e2**2))
    return np.sqrt(e1**2 + e2**2)

## When we want to fit multiple points (10% current recover, many times)